# Interactive notebook for plotting a map of the minimum subglacial lake size that is detectable by altimetry in Antarctica

This notebook accompanies a manuscript (in preparation) that explores the limitations of using altimetry to detect subglacial lake filling/draining events, quantify water volume change, and estimate lake lengths:
>Stubblefield, A. G., Creyts, T. T., Kingslake, J., Siegfried, M.R. & Spiegelman, M. (2021). Detectability of Active Subglacial Lakes by Altimetry. *In preparation for GRL*.

## Model description
### Governing equation
The main assumptions/approximations in deriving the governing equation are (1) ice flows as a linearly viscous fluid, (2) the basal sliding law is also linear (sliding velocity $\propto$ shear stress) , and (3) the initial domain is an infinite strip with finite thickness (the ice thickness).


The model is a small-perturbation approximation of the Stokes equations, where the perturbations are relative to the cryostatic state. The problem is solved via spatial Fourier transforms. We scale the spatial coordinate relative to the ice thickness $H$, time relative to the oscillation period $t_p$, and elevation relative to the oscillation amplitude $\mathcal{A}$. With this scaling, the transformed elevation evolves according to

$$ \frac{\partial \widehat{h}}{\partial t}=-\lambda\mathcal{R}\widehat{h}+ \mathcal{T}\widehat{w}_b $$

where 
- $\widehat{h}$ is the transformed elevation anomaly $h$, 
- $\widehat{w}_b$ is the transformed basal velocity anomaly $w_b$, 
- $\mathcal{T}$ is a base-to-surface transfer function, 
- $\mathcal{R}$ is a viscous relaxation function.

This equation is essentially an ODE in time (for each wavenumber), so $\widehat{h}$ can be solved for analytically (perhaps up to quadrature) in frequency space. The solution $h$ in physical space is then obtained via the inverse Fourier transform. 

### Nondimensional parameter 1: $\lambda$ 
The first parameter is

\begin{align}
\lambda = \frac{t_p}{t_r} 
\end{align}

where 

\begin{align}
t_r = \frac{4\pi\eta}{\rho_i g H}
\end{align}

is the characteristic relaxation time for harmonic perturbations with wavelength equal to the ice thickness $H$.
$\lambda$ controls the viscous relaxation of the upper surface. For small $\lambda$, the surface does not relax very much, leading to better correspondence between surface and base. For large $\lambda$, viscous relaxation spreads out and dampens the basal anomaly, leading to worse (or *no*) correspondence between surface and base.

### Nondimensional parameter 2: $\beta_\star$
The second parameter, hidden in the definitions of $\mathcal{R}$ and $\mathcal{T}$, is a nondimensional basal friction parameter:

$$\beta_\star = \frac{\beta H}{2\eta}, $$

where $\beta$ (Pa s/m) is the *dimensional* basal friction coefficient. High friction is associated with good correspondence between the surface and basal anomaly; low friction is associated with worse correspondence.

### Nondimensional parameter 3: $w_b$ 
Here, we consider basal velocity perturbations of the form

$$ w_b(x,t) = \exp\left(-\frac{x^2}{2\sigma^2}\right)\sin(t), $$

just a (spatial) Gaussian bump that oscillates in time. Note that time has been scaled by $t_p$, and vertical velocity has been scaled by $\mathcal{A}/t_p$. 

We define the length of the subglacial lake to be $L_\mathrm{true} = 6\sigma$, since the lake boundary coincides with where the vertical velocity anomaly approaches zero.

### Estimated lake length
We define the estimated lake length $L_\mathrm{est}$ as the length of the region where surface displacement exceeds some threshold:

$$ L_\mathrm{est} \equiv \text{length of region where } \Delta h > \delta_d\, / \,\mathcal{A},$$

where $\Delta h$ is the surface displacement between highstand and lowstand, and $\delta_d$ is a fixed dimensional displacement threhold (10 cm here).

### The main point is that  $L_\mathrm{est}$ can be ZERO (i.e., the lake is NOT detectable by altimetry) if... 
1. the ice is thick ($H$ large), 
2. the bed is slippery ($\beta$ small),
3. the oscillation is slow ($t_p$ large), or 
4. the oscillation is low-amplitude ($\mathcal{A}$ small)!

## Goal
The primary (dimensional) parameters we are interested in above are 
- Ice thickness $H$
- Basal friction coefficient $\beta$

...we have maps of how these vary across the Antarctic Ice Sheet! 

The free parameters that we can set are

- Oscillation amplitude $\mathcal{A}$
- Oscillation period $t_p$
- Subglacial lake length $L_\mathrm{true}$

### Smallest detectable subglacial lake length
For a given $t_p$ and $\mathcal{A}$, this notebook uses the ice thickness and friction maps to construct a map of the smallest lake length $L_\mathrm{true}$ that is detectable by altimetry. In other words, we define the minimum
detectable subglacial lake size via

$$L_\mathrm{min} \equiv \text{lake size where $L_\mathrm{est}\to 0$ transition occurs} $$

# Code

In [ ]:
# import everything...
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d
import copy
import xarray as xr
import fsspec
# define some functions
%run Functions.ipynb

## 1. Model parameters
Change the amplitude  `amp` ($\mathcal{A}$) and the oscillation period `t_pd` ($t_p$) to see the effect on the minimum detectable lake size

In [ ]:
amp = 1                                 # oscillation amplitude at base (m)
t_pd = 10*3.154e7                       # oscillation period (s)

delta = 0.1/amp                         # dimensionless displacement threshold corresponding
                                        # to dimensional threshold of 0.1 m

rho = 917.0                             # ice density kg/m^3
g = 9.81                                # gravitational acceleration m^2/s
eta = 1e12                              # constant viscosity (Pa s)

Ls = np.linspace(100,1,101)             # array of lake lengths (relative to ice thickness)
                                        # for computing the minimum detectable lake size
                                        # default minmium value = 1, maximum value = 100.

N_pts = 20                              # number of ice thickness and friction
                                        # values (between max and min values from data)
                                        # for constructing minimum lake size function

## 2. Load data
Next we load ice thickness and basal friction coefficient maps featured in the paper:
>Arthern, R. J., Hindmarsh, R. C., & Williams, C. R. (2015). Flow speed within the Antarctic ice sheet and its controls inferred from satellite observations. Journal of Geophysical Research: Earth Surface, 120(7), 1171-1188.

In [ ]:
H_beta_mapper = fsspec.get_mapper('gs://ldeo-glaciology/bedmachine/H_beta.zarr', mode='ab')
H_beta = xr.open_zarr(H_beta_mapper)  
H_beta.load()
Xv, Yv = np.meshgrid(H_beta.x,H_beta.y) # horizontal map coordinates
beta_d = H_beta.beta.data               # (dimensional) friction coefficient (Pa s / m)
H = H_beta.thickness.data               # ice thickness (m)

## 3. Compute the minimum detectable lake size as a function of $H$ and $\beta$

In [ ]:
# construct arrays for H and beta_d that cover the range of the data
H_int = np.linspace(1,np.max(H),N_pts)
beta_int = np.logspace(np.min(np.log10(beta_d)),np.max(np.log10(beta_d)),N_pts)

# array for minimum lake size at every (H,beta_d) value
min_Ls = np.zeros((np.size(H_int),np.size(beta_int)))

print('Computing minimum detectable lake size as function of friction and ice thickness....')

l = 0
for i in range(np.shape(min_Ls)[0]):
    for j in range(np.shape(min_Ls)[1]):
        min_Ls[i,j] = get_min_Ls(Ls,beta_int[i],H_int[j])
        if l % int(np.size(min_Ls)/10.0) == 0:
            print(str(100*l/int(np.size(min_Ls)))+' % complete')
        l+=1
print(str(100*l/int(np.size(min_Ls)))+' % complete')
print('\n')

MLS = interp2d(H_int,beta_int,min_Ls,kind='linear')

## 4. Create the map for Antarctica by evaluating the function from previous step


In [ ]:
Ls_map = np.zeros(np.shape(beta_d))     # minimum lake length map

print('Constructing map....')
l = 0
for i in range(np.shape(Ls_map)[0]):
    for j in range(np.shape(Ls_map)[1]):
        Ls_map[i,j] = MLS(H[i,j],beta_d[i,j])
        if l % int(np.size(Ls_map)/10.0) == 0:
            print(str(100*l/int(np.size(Ls_map)))+' % complete')
        l+=1
print(str(100*l/int(np.size(Ls_map)))+' % complete')
print('\n')

# mask out ice shelves (friction is nonzero there probably for inversion purposes)
Ls_map[beta_d<1e5] = 0

## 5. Plot maps of the smallest detectable lake length $L_\mathrm{min}$, ice thickness $H$, and friction coefficient $\beta$ for the Antarctic Ice Sheet

In [ ]:
levels_H = np.array([0,500,1000,1500,2000,2500,3000,3500,4000,4500])/1000.0
levels_beta = np.array([1e6,1e8,1e10,1e12,1e14,1e16])

# note: you may need to modify the contour levels (levels_L) for the minimum lake length
levels_L = np.array([0,5,10,15,20,25,30])


cmap1 = copy.copy(mpl.cm.get_cmap("Blues"))
cmap1.set_under('mistyrose')

plt.figure(figsize=(14,6))
plt.suptitle(r'$t_p$ = '+"{:.1f}".format(t_pd/3.154e7)+r' yr,   $\mathcal{A} =$ '+"{:.1f}".format(amp)+' m',fontsize=24)
plt.subplot(131)
p1 = plt.contourf(Xv/1000,Yv/1000,Ls_map*H/1000.0,cmap=cmap1,levels=levels_L,extend='both')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel(r'$x$',fontsize=16)
plt.ylabel(r'$y$',fontsize=16)
cbar = plt.colorbar(p1,orientation='horizontal')
cbar.set_label(r'$L_\mathrm{min}$ (km)',fontsize=20)


plt.subplot(132)
plt.xticks(fontsize=12)
plt.gca().yaxis.set_ticklabels([])
plt.xlabel(r'$x$',fontsize=16)
p1 = plt.contourf(Xv/1000,Yv/1000,H/1000.0,cmap=cmap1,levels=levels_H,extend='both')
cbar = plt.colorbar(p1,orientation='horizontal')
cbar.set_label(r'$H$ (km)',fontsize=20)

plt.subplot(133)
plt.xticks(fontsize=12)
plt.xlabel(r'$x$',fontsize=16)
plt.gca().yaxis.set_ticklabels([])
p2 = plt.contourf(Xv/1000,Yv/1000,beta_d,cmap=cmap1,levels=levels_beta,norm=mpl.colors.LogNorm(),extend='both')
cbar = plt.colorbar(p2,orientation='horizontal')
cbar.set_label(r'$\beta$ (Pa s / m)',fontsize=20)
plt.grid()
plt.tight_layout()